In [1]:
import sys
import os
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if path not in sys.path:
    sys.path.insert(0, path)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch

## Create dataset

In [3]:
torch.manual_seed(0)

class ToTensor(object):
    def __call__(self, image, target):
        image = torch.from_numpy(image / 255.0).float()
        image = image.permute((2, 0, 1))
        return image, target

class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

def get_transform():
    transforms = []
    #transforms.append(ToTensor()) 
    return Compose(transforms)

In [4]:
from rock_detection_2d.datasets.instance_segmentation.rgb_dataset import Dataset

In [5]:
dataset = Dataset('data/rocklas/tiles', 1000, transforms=get_transform(), include_name=False)
dataset_test = Dataset('data/rocklas/tiles', 1000, transforms=None, include_name=False)

In [6]:
print(len(dataset))
img, target = dataset[1]
print(target)
print(target['masks'].shape)
print(img.shape)

81
{'boxes': tensor([[182., 837., 285., 910.],
        [  0., 718., 185., 894.],
        [  0., 792., 185., 910.]]), 'labels': tensor([1, 1, 1]), 'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8), 'image_id': tensor([1]), 'area': tensor([ 7519., 32560., 21830.]), 'iscrowd': tensor([0, 0, 0])}
torch.Size([3, 1000, 1000])
torch.Size([3, 1000, 1

In [7]:
dataset.show(1)

In [8]:
pre_computed_stats = True
if not pre_computed_stats:
    data_mean, data_std, data_max, data_min = dataset.imageStat(100)
else:
    data_mean = [0.4508936958691256, 0.43597375552024875, 0.3957421697130724]
    data_std = [0.2869340501454762, 0.27112731116220695, 0.24573479879090088]
    data_max = [1.0, 1.0, 1.0]
    data_min = [0.0, 0.0, 0.0]
print(data_mean)
print(data_std)
print(data_max)
print(data_min)

[0.4508936958691256, 0.43597375552024875, 0.3957421697130724]
[0.2869340501454762, 0.27112731116220695, 0.24573479879090088]
[1.0, 1.0, 1.0]
[0.0, 0.0, 0.0]


In [9]:
# define training and validation data loaders
import utils
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=4, shuffle=False, num_workers=8,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

## Train mask rcnn

In [10]:

import math
import sys
import time
import torch

import torchvision.models.detection.mask_rcnn

from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator
import utils


def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = utils.warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print(loss_dict_reduced)
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])


def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types


@torch.no_grad()
def evaluate(model, data_loader, device):
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Test:'

    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)

    for image, targets in metric_logger.log_every(data_loader, 100, header):
        image = list(img.to(device) for img in image)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(image)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator

In [11]:
from rock_detection_2d.models.mask_rcnn import get_model_instance_segmentation

In [12]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cuda:0')

In [13]:
num_classes = 2
input_c = 3
detections_per_img = 256
anchor_size = ((16,), (32,), (64,), (128,), (256,))

In [14]:
mask_rcnn = get_model_instance_segmentation(num_classes, input_channel=input_c, image_mean=data_mean, image_std=data_std, stats=True, detections_per_img=detections_per_img, anchor_size=anchor_size)

In [15]:
mask_rcnn.to(device)
params = [p for p in mask_rcnn.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=8,
                                               gamma=0.5)
init_epoch = 0
num_epochs = 100

In [16]:
save_param = "model/mask_rcnn/epoch_{:04d}.param".format(init_epoch)
torch.save(mask_rcnn.state_dict(), save_param)


for epoch in range(init_epoch, init_epoch + num_epochs):
    save_param = "model/mask_rcnn/epoch_{:04d}.param".format(epoch)
    #torch.save(mask_rcnn.state_dict(), save_param)
    # train for one epoch, printing every 10 iterations
    print(save_param)
    train_one_epoch(mask_rcnn, optimizer, data_loader, device, epoch, print_freq=100)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    #print('\n')
    #print("trained_param_4/epoch_00%02d.param" % epoch)
    #mask_rcnn.load_state_dict(torch.load("trained_param_4/epoch_00%02d.param" % epoch))
    evaluate(mask_rcnn, data_loader_test, device=device)

    #save_param = "trained_param_8_fresh/epoch_{:04d}.param".format(epoch)
    torch.save(mask_rcnn.state_dict(), save_param)

model/mask_rcnn/epoch_0000.param


/home/dreamslab_legion/.local/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: [0]  [ 0/21]  eta: 0:00:39  lr: 0.000510  loss: 3.3086 (3.3086)  loss_classifier: 0.9712 (0.9712)  loss_box_reg: 0.0581 (0.0581)  loss_mask: 2.1354 (2.1354)  loss_objectness: 0.1332 (0.1332)  loss_rpn_box_reg: 0.0106 (0.0106)  time: 1.8805  data: 0.6887  max mem: 3074
Epoch: [0]  [20/21]  eta: 0:00:01  lr: 0.010000  loss: 1.0646 (1.4673)  loss_classifier: 0.1180 (0.2249)  loss_box_reg: 0.0557 (0.0607)  loss_mask: 0.6793 (1.0576)  loss_objectness: 0.0694 (0.1077)  loss_rpn_box_reg: 0.0095 (0.0164)  time: 1.1639  data: 0.0092  max mem: 3364
Epoch: [0] Total time: 0:00:25 (1.1997 s / it)
creating index...
index created!
Test:  [ 0/81]  eta: 0:00:31  model_time: 0.1538 (0.1538)  evaluator_time: 0.0297 (0.0297)  time: 0.3855  data: 0.1991  max mem: 3364
Test:  [80/81]  eta: 0:00:00  model_time: 0.2347 (0.2550)  evaluator_time: 0.1271 (0.1468)  time: 0.4237  data: 0.0027  max mem: 3364
Test: Total time: 0:00:33 (0.4142 s / it)
Averaged stats: model_time: 0.2347 (0.2550)  evaluator_tim

model/mask_rcnn/epoch_0003.param
Epoch: [3]  [ 0/21]  eta: 0:00:44  lr: 0.010000  loss: 0.8041 (0.8041)  loss_classifier: 0.0987 (0.0987)  loss_box_reg: 0.0580 (0.0580)  loss_mask: 0.5966 (0.5966)  loss_objectness: 0.0440 (0.0440)  loss_rpn_box_reg: 0.0069 (0.0069)  time: 2.1145  data: 0.7436  max mem: 3365
Epoch: [3]  [20/21]  eta: 0:00:01  lr: 0.010000  loss: 0.8113 (0.8209)  loss_classifier: 0.1034 (0.1142)  loss_box_reg: 0.0850 (0.0917)  loss_mask: 0.5637 (0.5603)  loss_objectness: 0.0303 (0.0434)  loss_rpn_box_reg: 0.0082 (0.0114)  time: 1.2517  data: 0.0078  max mem: 3365
Epoch: [3] Total time: 0:00:27 (1.2948 s / it)
creating index...
index created!
Test:  [ 0/81]  eta: 0:00:46  model_time: 0.2505 (0.2505)  evaluator_time: 0.1185 (0.1185)  time: 0.5787  data: 0.2067  max mem: 3365
Test:  [80/81]  eta: 0:00:00  model_time: 0.2868 (0.2811)  evaluator_time: 0.1809 (0.1679)  time: 0.4975  data: 0.0027  max mem: 3365
Test: Total time: 0:00:37 (0.4626 s / it)
Averaged stats: model_tim

model/mask_rcnn/epoch_0006.param
Epoch: [6]  [ 0/21]  eta: 0:00:39  lr: 0.010000  loss: 0.7749 (0.7749)  loss_classifier: 0.0851 (0.0851)  loss_box_reg: 0.0770 (0.0770)  loss_mask: 0.5726 (0.5726)  loss_objectness: 0.0337 (0.0337)  loss_rpn_box_reg: 0.0065 (0.0065)  time: 1.8988  data: 0.6494  max mem: 3365
Epoch: [6]  [20/21]  eta: 0:00:01  lr: 0.010000  loss: 0.7186 (0.7326)  loss_classifier: 0.0964 (0.0991)  loss_box_reg: 0.1005 (0.1065)  loss_mask: 0.4775 (0.4825)  loss_objectness: 0.0257 (0.0342)  loss_rpn_box_reg: 0.0082 (0.0104)  time: 1.2561  data: 0.0084  max mem: 3365
Epoch: [6] Total time: 0:00:27 (1.2887 s / it)
creating index...
index created!
Test:  [ 0/81]  eta: 0:00:28  model_time: 0.1404 (0.1404)  evaluator_time: 0.0087 (0.0087)  time: 0.3518  data: 0.2001  max mem: 3365
Test:  [80/81]  eta: 0:00:00  model_time: 0.2059 (0.2035)  evaluator_time: 0.0804 (0.0789)  time: 0.3063  data: 0.0026  max mem: 3365
Test: Total time: 0:00:23 (0.2926 s / it)
Averaged stats: model_tim

...

model/mask_rcnn/epoch_0087.param
Epoch: [87]  [ 0/21]  eta: 0:00:47  lr: 0.000010  loss: 0.2040 (0.2040)  loss_classifier: 0.0234 (0.0234)  loss_box_reg: 0.0474 (0.0474)  loss_mask: 0.1279 (0.1279)  loss_objectness: 0.0021 (0.0021)  loss_rpn_box_reg: 0.0031 (0.0031)  time: 2.2517  data: 0.8006  max mem: 3405
Epoch: [87]  [20/21]  eta: 0:00:01  lr: 0.000010  loss: 0.1443 (0.1680)  loss_classifier: 0.0169 (0.0220)  loss_box_reg: 0.0255 (0.0399)  loss_mask: 0.0931 (0.1002)  loss_objectness: 0.0021 (0.0025)  loss_rpn_box_reg: 0.0022 (0.0034)  time: 1.3175  data: 0.0075  max mem: 3405
Epoch: [87] Total time: 0:00:28 (1.3638 s / it)
creating index...
index created!
Test:  [ 0/81]  eta: 0:00:28  model_time: 0.1365 (0.1365)  evaluator_time: 0.0107 (0.0107)  time: 0.3516  data: 0.2015  max mem: 3405
Test:  [80/81]  eta: 0:00:00  model_time: 0.1356 (0.1420)  evaluator_time: 0.0093 (0.0151)  time: 0.1657  data: 0.0027  max mem: 3405
Test: Total time: 0:00:13 (0.1651 s / it)
Averaged stats: model_

model/mask_rcnn/epoch_0090.param
Epoch: [90]  [ 0/21]  eta: 0:00:44  lr: 0.000005  loss: 0.2090 (0.2090)  loss_classifier: 0.0267 (0.0267)  loss_box_reg: 0.0474 (0.0474)  loss_mask: 0.1279 (0.1279)  loss_objectness: 0.0039 (0.0039)  loss_rpn_box_reg: 0.0031 (0.0031)  time: 2.1230  data: 0.7630  max mem: 3405
Epoch: [90]  [20/21]  eta: 0:00:01  lr: 0.000005  loss: 0.1426 (0.1679)  loss_classifier: 0.0159 (0.0221)  loss_box_reg: 0.0255 (0.0396)  loss_mask: 0.0929 (0.1001)  loss_objectness: 0.0017 (0.0026)  loss_rpn_box_reg: 0.0022 (0.0034)  time: 1.3178  data: 0.0082  max mem: 3405
Epoch: [90] Total time: 0:00:28 (1.3580 s / it)
creating index...
index created!
Test:  [ 0/81]  eta: 0:00:28  model_time: 0.1365 (0.1365)  evaluator_time: 0.0108 (0.0108)  time: 0.3524  data: 0.2018  max mem: 3405
Test:  [80/81]  eta: 0:00:00  model_time: 0.1355 (0.1421)  evaluator_time: 0.0093 (0.0152)  time: 0.1661  data: 0.0027  max mem: 3405
Test: Total time: 0:00:13 (0.1654 s / it)
Averaged stats: model_

model/mask_rcnn/epoch_0093.param
Epoch: [93]  [ 0/21]  eta: 0:00:47  lr: 0.000005  loss: 0.2083 (0.2083)  loss_classifier: 0.0276 (0.0276)  loss_box_reg: 0.0474 (0.0474)  loss_mask: 0.1279 (0.1279)  loss_objectness: 0.0023 (0.0023)  loss_rpn_box_reg: 0.0031 (0.0031)  time: 2.2483  data: 0.8302  max mem: 3405
Epoch: [93]  [20/21]  eta: 0:00:01  lr: 0.000005  loss: 0.1441 (0.1676)  loss_classifier: 0.0159 (0.0223)  loss_box_reg: 0.0255 (0.0396)  loss_mask: 0.0929 (0.0999)  loss_objectness: 0.0018 (0.0024)  loss_rpn_box_reg: 0.0022 (0.0034)  time: 1.3181  data: 0.0077  max mem: 3405
Epoch: [93] Total time: 0:00:28 (1.3642 s / it)
creating index...
index created!
Test:  [ 0/81]  eta: 0:00:28  model_time: 0.1372 (0.1372)  evaluator_time: 0.0120 (0.0120)  time: 0.3540  data: 0.2018  max mem: 3405
Test:  [80/81]  eta: 0:00:00  model_time: 0.1357 (0.1422)  evaluator_time: 0.0093 (0.0151)  time: 0.1658  data: 0.0027  max mem: 3405
Test: Total time: 0:00:13 (0.1653 s / it)
Averaged stats: model_

model/mask_rcnn/epoch_0096.param
Epoch: [96]  [ 0/21]  eta: 0:00:48  lr: 0.000002  loss: 0.2065 (0.2065)  loss_classifier: 0.0260 (0.0260)  loss_box_reg: 0.0474 (0.0474)  loss_mask: 0.1278 (0.1278)  loss_objectness: 0.0022 (0.0022)  loss_rpn_box_reg: 0.0031 (0.0031)  time: 2.3227  data: 0.7927  max mem: 3405
Epoch: [96]  [20/21]  eta: 0:00:01  lr: 0.000002  loss: 0.1433 (0.1684)  loss_classifier: 0.0177 (0.0229)  loss_box_reg: 0.0255 (0.0398)  loss_mask: 0.0929 (0.1000)  loss_objectness: 0.0017 (0.0022)  loss_rpn_box_reg: 0.0022 (0.0034)  time: 1.3181  data: 0.0074  max mem: 3405
Epoch: [96] Total time: 0:00:28 (1.3678 s / it)
creating index...
index created!
Test:  [ 0/81]  eta: 0:00:28  model_time: 0.1363 (0.1363)  evaluator_time: 0.0107 (0.0107)  time: 0.3514  data: 0.2012  max mem: 3405
Test:  [80/81]  eta: 0:00:00  model_time: 0.1357 (0.1421)  evaluator_time: 0.0094 (0.0151)  time: 0.1659  data: 0.0026  max mem: 3405
Test: Total time: 0:00:13 (0.1652 s / it)
Averaged stats: model_

model/mask_rcnn/epoch_0099.param
Epoch: [99]  [ 0/21]  eta: 0:00:48  lr: 0.000002  loss: 0.2052 (0.2052)  loss_classifier: 0.0234 (0.0234)  loss_box_reg: 0.0473 (0.0473)  loss_mask: 0.1278 (0.1278)  loss_objectness: 0.0036 (0.0036)  loss_rpn_box_reg: 0.0031 (0.0031)  time: 2.3187  data: 0.7857  max mem: 3405
Epoch: [99]  [20/21]  eta: 0:00:01  lr: 0.000002  loss: 0.1423 (0.1683)  loss_classifier: 0.0179 (0.0228)  loss_box_reg: 0.0254 (0.0395)  loss_mask: 0.0928 (0.1001)  loss_objectness: 0.0018 (0.0025)  loss_rpn_box_reg: 0.0022 (0.0034)  time: 1.3171  data: 0.0074  max mem: 3405
Epoch: [99] Total time: 0:00:28 (1.3666 s / it)
creating index...
index created!
Test:  [ 0/81]  eta: 0:00:28  model_time: 0.1363 (0.1363)  evaluator_time: 0.0109 (0.0109)  time: 0.3552  data: 0.2044  max mem: 3405
Test:  [80/81]  eta: 0:00:00  model_time: 0.1361 (0.1418)  evaluator_time: 0.0092 (0.0150)  time: 0.1652  data: 0.0027  max mem: 3405
Test: Total time: 0:00:13 (0.1648 s / it)
Averaged stats: model_